# Running Counter

In [ ]:
# AGENCY CODE

import numpy as np

class Agency:
    """
    This class acts as the service, also known as the agency,
    which holds the secret vector x and responds with the vector, plus some random noise
    """
    def __init__(self, n):
        """
        n: the size of the secret binary vector (length = number of entries)
        sigma: standard deviation of gaussian noise in each response
        """
        
        self._n = n
        self._x = np.random.randint(2, size=self._n)
        self._z = np.random.randint(2, size=self._n)
    
    @property
    def n(self):
        return self._n
    
    def get_extra_information(mu):
        """
        For each element of the secret vector, generate a Bernoulli random variable, 
        returning x with probability mu and 1 - x with probability 1 - mu
        
        Input
            mu: probability of true answer
        Output:
            a vector representing a probabilistic view of x
        """
        return np.vectorize(lambda xi, mu: xi if random.random() > (1 - mu) else 1 - xi)(x, mu)
    
    def query(self):
        """
        Input
            None (there is only one query allowed, which agency will generate)
        Output
            1/n B @ x, with gaussian noise added, mean 0, stddev sigma
        """
        
        return np.tril(np.ones((self._n, self._n))) @ self._x + self._z
    
    def guess(self, x1):
        """
        Input
            x1: Vector you guess to be secret vector
        Output
            Honest answer of fraction of correct bits in x
        """
        return np.count_nonzero(self._x == x1) / self._n

In [ ]:
# ATTACKER CODE

def attack_running_counter(agcy, get_extra):
    # Get the query output and make a workspace to guess the bits
    w1 = agcy.query()
    x1 = np.random.randint(2, size=agcy.n) # Initialize to random; any undeducible bits will guessed randomly
    mask = np.zeros(agcy.n)                # Bit vector marking which bits we have DEDUCED (not guessed)
    
    # Half the time we can immediately deduce the first bit, so do so if possible
    if w1[0] == 0:
        # Since the error is only positive, 0 means both x0 and z0 are 0. Score!
        x1[0] = 0
        mask[0] = 1;
    elif w1[0] == 2:
        # Since we have only seen 1 user but have reported 2, we know that both x0 and z0 are 1. Score!
        x1[0] = 1
        mask[0] = 1;
    
    # Deduce bit i based on what is known of bit i - 1
    for i in range(1, agcy.n):
        # Subtract w[i-1] from w[i]
        # case -1: x[i] = 0
        # case 0: ambiguous
        # case 1: ambiguous
        # case 2: x[i] = 1
        delta = w1[i] - w1[i - 1]
        # Obvious cases
        if delta == -1:
            # Our counter went down; this can only mean that z[i-1] = 1 and x[i] and z[i] = 0
            x1[i] = 0
            mask[i] = 1
        elif delta == 2:
            # Our counter went up by 2; this can only mean that z[i-1] = 0 and x[i] and z[i] = 1
            x1[i] = 1
            mask[i] = 1
        #Subtle cases
        elif delta == 0:
            # Check if we know the last bit
            if mask[i - 1] == 1:
                # Our counter increased, but we know that both x[i-1] and z[i-1] equal 0
                # Therefore both x[i] and z[i] also equal 0
                x1[i] = 0
                mask[i] = 1
            # Default to leaving random guess; at this point either x[i] = 1 or z[i] = 1, each with 1/2 probability
        elif delta == 1:
            # Check if we know the last bit
            if mask[i - 1] == 1:
                # Our counter increased, but we know that both x[i-1] and z[i-1] equal 1
                # Therefore both x[i] and z[i] also equal 1
                x1[i] = 1
                mask[i] = 1
            # Default to leaving random guess; at this point either x[i] = 1 or z[i] = 1, each with 1/2 probability
    
    # Submit our guess and report the score
    return agcy.guess(x1)

In [ ]:
# TEST SUITE

n_values = [100, 500, 1000, 5000]

for n in n_values:
    ag = Agency(n)
    print(attack_running_counter(ag, False))
